In [14]:
import pandas as pd

data = pd.read_csv('data/data.csv')  # Weather data from BH
data.head()

,Date,Average Temperature,Minimum Temperature,Maximum Temperature,Total Precipitation,wdir,Average Wind Speed,Sea-Level Air Pressure
0,2021-01-02,22.9,19.1,29.0,7.5,80.0,8.5,1014.7
1,2021-01-03,21.8,18.0,28.0,25.9,77.0,9.2,1014.0
2,2021-01-04,21.4,17.9,27.0,7.9,70.0,8.8,1015.8
3,2021-01-05,23.8,18.5,32.0,2.7,82.0,7.9,1016.7
4,2021-01-06,23.5,19.3,32.0,2.8,100.0,8.6,1015.8


In [16]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

# Preparar dados
df = data.copy()
df.Date = pd.to_datetime(df.Date.values)
df.set_index('Date', inplace=True)

# Escolher os parâmetros p, d, q
# (neste exemplo, usaremos p=2, d=1 e q=1, mas é recomendável usar métodos como AIC ou BIC para escolher os melhores valores)
p, d, q = 2, 1, 1

# Define a frequência dos dados para diária (D)
freq = 'D'

# Treinar o modelo ARIMA
model = ARIMA(df, order=(p, d, q), freq=freq)
model_fit = model.fit()

# Realizar previsões para os próximos 12 meses
n_periods = 12
forecast = model_fit.forecast(steps=n_periods, alpha=0.05, freq=freq)

# Criar um índice de data para as previsões
forecast_index = pd.date_range(start='2023-01-01', periods=n_periods, freq=freq)

forecast

ValueError: SARIMAX models require univariate `endog`. Got shape (808, 7).